In [1]:
import pandas as pd 
import numpy as np
import jieba
from gensim.models.word2vec import Word2Vec
from gensim.models.word2vec import LineSentence
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import os
from bs4 import BeautifulSoup
from hanziconv import HanziConv
from tqdm import tqdm_notebook as tqdm
import re
from collections import defaultdict
import logging
import warnings
warnings.simplefilter('ignore')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(massage)s',level=logging.ERROR)

/Users/tracy/anaconda3/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
#读新闻内容
news_content = pd.read_csv('/Users/tracy/GithubFiles/sqlResult_1558435.csv')['content'].tolist()

In [3]:
news_content = news_content[:40000]

In [4]:
wiki_path = '/Users/tracy/GithubFiles/wiki_dump/AA/'
wiki_list = os.listdir(wiki_path)
# wiki_list.sort()

In [5]:
wiki_content = []
for path in wiki_list[:10]:
    with open( wiki_path + path ,'r',encoding='utf-8') as f:
        lines = f.read()
        lines = BeautifulSoup(lines,'lxml')
        docs = lines.find_all('doc')
        for d in docs:
            d_text = d.text
            wiki_content.append(d_text)

In [6]:
content = news_content + wiki_content

In [7]:
content = [HanziConv.toSimplified(x) for x in tqdm(content) if type(x) == str and len(x)>5 ]

### 清洗

In [8]:
def token(string):
    return ' '.join(re.findall('[\w]+',string))

In [9]:
content_1 = [token(x) for x in content]

In [10]:
content[0]

'此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\n'

### 分词

In [11]:
tokens_1 = [ ' '.join(jieba.lcut(x)) for x in tqdm(content_1) ]

Building prefix dict from the default dictionary ...
--- Logging error ---
Traceback (most recent call last):
  File "/Users/tracy/anaconda3/lib/python3.6/logging/__init__.py", line 994, in emit
    msg = self.format(record)
  File "/Users/tracy/anaconda3/lib/python3.6/logging/__init__.py", line 840, in format
    return fmt.format(record)
  File "/Users/tracy/anaconda3/lib/python3.6/logging/__init__.py", line 580, in format
    s = self.formatMessage(record)
  File "/Users/tracy/anaconda3/lib/python3.6/logging/__init__.py", line 549, in formatMessage
    return self._style.format(record)
  File "/Users/tracy/anaconda3/lib/python3.6/logging/__init__.py", line 391, in format
    return self._fmt % record.__dict__
KeyError: 'massage'
Call stack:
  File "/Users/tracy/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/tracy/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/tracy/anac

Loading model cost 0.713 seconds.
--- Logging error ---
Traceback (most recent call last):
  File "/Users/tracy/anaconda3/lib/python3.6/logging/__init__.py", line 994, in emit
    msg = self.format(record)
  File "/Users/tracy/anaconda3/lib/python3.6/logging/__init__.py", line 840, in format
    return fmt.format(record)
  File "/Users/tracy/anaconda3/lib/python3.6/logging/__init__.py", line 580, in format
    s = self.formatMessage(record)
  File "/Users/tracy/anaconda3/lib/python3.6/logging/__init__.py", line 549, in formatMessage
    return self._style.format(record)
  File "/Users/tracy/anaconda3/lib/python3.6/logging/__init__.py", line 391, in format
    return self._fmt % record.__dict__
KeyError: 'massage'
Call stack:
  File "/Users/tracy/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/tracy/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/tracy/anaconda3/lib/python3.6

In [14]:
tokens_1 = [x.replace('  ',' ') for x in tokens_1]

### Normalize 转化成词向量 

In [15]:
#先保存处理好的句子
with open('content_cut.txt',mode='w',encoding='utf-8') as f:
    for t in tqdm(tokens_1):
        f.write(t +'\n')

In [16]:
sentences_1 = LineSentence('content_cut.txt')

In [17]:
model_1 = Word2Vec(sentences_1, min_count=10,size=100,workers=4)

In [119]:
model_1.wv.most_similar('说',topn=10)

[('表示', 0.7852936387062073),
 ('指出', 0.7177790403366089),
 ('看来', 0.67959064245224),
 ('坦言', 0.6606078743934631),
 ('认为', 0.6565666198730469),
 ('告诉', 0.6537516117095947),
 ('称', 0.6389726400375366),
 ('明说', 0.6327576041221619),
 ('说道', 0.6305349469184875),
 ('写道', 0.6010029315948486)]

In [19]:
def get_similar_words(initial_words,model):
    unseen = initial_words
    seen =  defaultdict(int)
    
    max_size = 500
    
    while unseen and len(seen) < max_size:
        
        #node = unseen.pop(0)[0]
        node = unseen.pop(0)
        
        #new_expanding = [ (w,s) for w, s in model.wv.most_similar(node,topn=20)]
        new_expanding = [ w for w, s in model.wv.most_similar(node,topn=20) ]
        
        unseen += new_expanding
        
        #seen[node] += new_expanding[1]
        seen[node] += 1
       
    return seen

In [139]:
from collections import defaultdict 
node_dict = defaultdict(int)

In [140]:
node_dict

defaultdict(int, {})

In [196]:
# sum_similar = 0 
# def get_similar_words_2(target,model): #需要获得 说 最多的搜索路径
    
#     wv = model.wv

#     similar_word_list = wv.most_similar(target,topn=10) 

#     similar_word_list = [x[0] for x in similar_word_list]
   
#     find_target = lambda word: (word,1) if word not in node_dict.keys() \
#         and '说' in [y[0] for y in wv.most_similar(word,topn=10)] else None
    
#     next_level = [ h(s) for s in similar_word_list]
    
#     print(next_level)
    
#     sum_similar = sum([x[1] for x in next_level])
#     print('sum_similar',sum_similar)
    
# #     print(sum_similar)
#     if sum_similar <= 1:
#         return 0
#     else:
#         for w,s in next_level:
#             node_dict[w] += s    
# #         print(node_dict)
#             get_similar_words_2(w,model_1)
            
        
        

In [194]:
get_similar_words_2('说',model_1)

[('表示', 1), ('指出', 1), ('看来', 1), ('坦言', 1), ('认为', 1), ('告诉', 1), ('称', 1), ('明说', 1), ('说道', 0), ('写道', 1)]
sum_similar 9


NameError: name 'next_similar' is not defined

In [168]:
node_dict

defaultdict(int,
            {'表示': 13,
             '指出': 20,
             '看来': 20,
             '坦言': 20,
             '认为': 20,
             '告诉': 13,
             '称': 20,
             '明说': 13,
             '说道': 0,
             '写道': 13,
             '透露': 7,
             '强调': 0,
             '声称': 0,
             '宣称': 7})

In [115]:
similar_words_1 = get_similar_words(['说','表示'],model_1)

In [116]:
sorted(similar_words_1.items(),key=lambda x: x[1],reverse=True)[:20]

[('说', 70),
 ('指出', 62),
 ('表示', 55),
 ('认为', 45),
 ('透露', 45),
 ('声称', 45),
 ('坦言', 41),
 ('说道', 41),
 ('宣称', 38),
 ('看来', 35),
 ('告诉', 35),
 ('强调', 34),
 ('写道', 32),
 ('相信', 30),
 ('觉得', 28),
 ('称', 26),
 ('否认', 26),
 ('地说', 25),
 ('指责', 25),
 ('直言', 24)]